In [6]:
import psycopg2

In [21]:
try:
    conn = psycopg2.connect("dbname='postgres' user='postgres' host='localhost' password='password'")
except:
    print("I am unable to connect to the database")

I am unable to connect to the database


In [22]:
# we use a context manager to scope the cursor session
with conn.cursor() as curs:

    try:
        # simple single row system query
        curs.execute("SELECT version()")

        # returns a single row as a tuple
        single_row = curs.fetchone()

        # use an f-string to print the single tuple returned
        print(f"{single_row}")

        # simple multi row system query
        curs.execute("SELECT query, backend_type FROM pg_stat_activity")

        # a default install should include this query and some backend workers
        many_rows = curs.fetchmany(5)

        # use the * unpack operator to print many_rows which is a Python list
        print(*many_rows, sep = "\n")

    # a more robust way of handling errors
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

('PostgreSQL 13.16 (Ubuntu 13.16-1.pgdg22.04+1) on aarch64-unknown-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)
('', 'autovacuum launcher')
('', 'logical replication launcher')
('', 'TimescaleDB Background Worker Launcher')
('', 'TimescaleDB Background Worker Scheduler')
('', 'TimescaleDB Background Worker Scheduler')


In [23]:
record = {
    'time': '2005-10-19 10:24:56+02',
    'stream_id': 'http://buildsys.org/ontologies/BLDG1#BLDG1.AHU.AHU01A.CCV',
    'value': 30.0,
}

In [24]:
with conn.cursor() as curs:

    try:
        # simple multi row system query
        curs.execute("""
                     INSERT INTO data ( time, stream_id, value ) 
                     VALUES (%(time)s, %(stream_id)s, %(value)s);
                     """, record)

    # a more robust way of handling errors
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

conn.commit()

insert or update on table "_hyper_1_59_chunk" violates foreign key constraint "59_106_data_stream_id_fkey"
DETAIL:  Key (stream_id)=(http://buildsys.org/ontologies/BLDG1#BLDG1.AHU.AHU01A.CCV) is not present in table "streams".



In [25]:
conn.close()

In [7]:
from pathlib import Path
import pickle
import zipfile
import pandas as pd

In [8]:
data_zip_file = '../../datasets/bts_site_b_train/train.zip'
mapping_csv_file = '../../datasets/bts_site_b_train/mapper_TrainOnly.csv'

In [9]:
try:
    conn = psycopg2.connect("dbname='buildingiot' user='buildingiot' host='localhost' password='buildingiotpassword'")
except:
    print("I am unable to connect to the database")

In [11]:

from io import StringIO


data_zip_path = Path(data_zip_file)
mapping_path = Path(mapping_csv_file)
mapping_df = pd.read_csv(mapping_path, index_col=0)

# Mappings for building B only, and ignore streams not saved to file
mapping_df = mapping_df[mapping_df['Building'] == 'B']
mapping_df = mapping_df[mapping_df['Filename'].str.contains('FILE NOT SAVED') == False]

# first = True
# better to move this outside the loop and have fewer commits?
# or does that risk losing data if something goes wrong?
with conn.cursor() as curs:
    i=0
    with zipfile.ZipFile(data_zip_path, 'r') as db_zip:
        for path in db_zip.namelist():
            if not path.endswith('.pkl'):
                continue

            filename = Path(path).name
            record = mapping_df.loc[mapping_df['Filename'] == filename, 'StreamID']

            # ignore streams that don't have a mapping
            if record.empty:
                continue

            i+=1

            # if first:
            #     first = False
            #     continue

            site = 'bts-b'
            stream_id = record.iloc[0]
            brick_class = mapping_df.loc[mapping_df['Filename'] == filename, 'strBrickLabel'].iloc[0]

            stream = {
                'id': stream_id,
                'site': site,
                'brick_class': brick_class,
            }

            # try:
            #     # simple multi row system query
            #     curs.execute("""
            #                  INSERT INTO streams ( id, site, brick_class ) 
            #                  VALUES (%(id)s, %(site)s, %(brick_class)s);
            #                  """, stream)
            # except (Exception, psycopg2.DatabaseError) as error:
            #     print(error)
            #     continue

            pkl_data = db_zip.read(path)
            data = pickle.loads(pkl_data)
            data_df = pd.DataFrame(data)
            data_df.drop(columns=['y'], inplace=True)
            data_df = data_df.astype({'t': 'str', 'v': 'float64'})
            data_df['stream_id'] = stream_id
            # print(data_df.head())
            buffer = StringIO()
            data_df.to_csv(buffer, index=False, header=False)
            buffer.seek(0)

            try:
                # simple multi row system query
                # curs.execute("""
                #                 INSERT INTO data ( time, stream_id, value ) 
                #                 VALUES (%(time)s, %(stream_id)s, %(value)s);
                #                 """, data)
                curs.execute("""
                            INSERT INTO streams ( id, site, brick_class ) 
                            VALUES (%(id)s, %(site)s, %(brick_class)s);
                            """, stream)
                curs.copy_from(buffer, 'data', columns=('time', 'value', 'stream_id'), sep=",")
            except (Exception, psycopg2.DatabaseError) as error:
                print(error)
                continue

            # FIXME: would adding a commit here improve performance?
            # conn.commit()
            print(i, f"Inserted {len(data_df)} records for {stream_id}")

            # print(buffer.read())
            # break
            # data_records = data_df.to_dict('records')
            # data_records = data_df.to_records()
            # print(data_records[0])
            # print([type(x) for x in data_records[0]])
            # tuples = [tuple(x) for x in data_df.to_numpy()]
            # print(tuples[0])
            # print([type(x) for x in tuples[0]])
            # break

            # for t, v in zip(data['t'], data['v']):
            #     data = {
            #         'time': t,
            #         'stream_id': stream_id,
            #         'value': v,
            #     }
            #     print(data)
            #     break

            # for t, v, _ in tuples:
            #     data = {
            #         'time': t,
            #         'stream_id': stream_id,
            #         'value': v,
            #     }


            # with conn.cursor() as curs:
            #     try:
            #         # simple multi row system query
            #         curs.execute("""
            #                         INSERT INTO data ( time, stream_id, value ) 
            #                         VALUES (%(time)s, %(stream_id)s, %(value)s);
            #                         """, data)
            #     except (Exception, psycopg2.DatabaseError) as error:
            #         print(error)
            #         continue

            # conn.commit()
            # print(f"Inserted {len(tuples)} records for {stream_id}")


/var/folders/_9/s1xwzrv537n2xj30q_b98s380000gn/T/ipykernel_12862/322298294.py:56: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.loads(pkl_data)


1 Inserted 51193 records for 6f502ba0_77fb_43db_b6dc_48e68d5c822b


/var/folders/_9/s1xwzrv537n2xj30q_b98s380000gn/T/ipykernel_12862/322298294.py:56: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.loads(pkl_data)


2 Inserted 51194 records for 3c9ca09b_8952_45da_a066_bcb645cbbc68


/var/folders/_9/s1xwzrv537n2xj30q_b98s380000gn/T/ipykernel_12862/322298294.py:56: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.loads(pkl_data)


...

666 Inserted 51202 records for 59195fd8_9b97_4fa2_b0ab_30effed9cf62


/var/folders/_9/s1xwzrv537n2xj30q_b98s380000gn/T/ipykernel_12862/322298294.py:56: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.loads(pkl_data)


667 Inserted 51194 records for adb30815_faf6_4590_95a7_c4c0f7bfd3d9


/var/folders/_9/s1xwzrv537n2xj30q_b98s380000gn/T/ipykernel_12862/322298294.py:56: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.loads(pkl_data)


668 Inserted 51190 records for 166e852b_0e15_4fa9_9a6c_a4fe3d04bf39


In [12]:
conn.commit()

In [13]:
conn.close()